In [2]:
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import glob

/home/kaih/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
train = pd.read_csv('../../../data/raw/train.csv')
test = pd.read_csv('../../../data/raw/sample_submission_zero.csv')
members = pd.read_csv('../../../data/raw/members.csv')

In [10]:
aggtrain = pd.read_csv('../../results/aggtrain.csv',header=0)

In [11]:
aggtest = pd.read_csv('../../results/aggtest.csv')

In [12]:
aggtrain =aggtrain.fillna(0)
aggtest=aggtest.fillna(0)

In [17]:
continuous_features = [
    'bd',
                       'avg(payment_plan_days)',
                       'avg(plan_list_price)',
                       'avg(actual_amount_paid)',
                       'avg(num_100)',
                       'avg(num_25)',
                       'avg(num_75)',
                       'avg(num_unq)',
                       'avg(num_50)',
                       'avg(num_985)',
                       'avg(total_secs)'
]
categorical_features = [
    'city',
                       'gender',
                       'registered_via',
                       '_c0',
                       'max(is_cancel)',
                       'min(is_auto_renew)'
]


In [18]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
le = LabelEncoder()
for i in categorical_features:
    aggtrain[i] = le.fit_transform(aggtrain[i].apply(str).values)
    aggtest[i] = le.fit_transform(aggtest[i].apply(str).values)

In [19]:
X = aggtrain[categorical_features+continuous_features]
y = aggtrain.is_churn

In [20]:
from sklearn.metrics import log_loss
def xgb_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'log_loss', log_loss(labels, preds)

In [25]:
fold = 3
round = 0
for i in np.random.randint(1,1000,fold):

    params = {
    #     'eta': 0.05,
        'learning_rate': 0.05, # use smaller learning rate to prevent overfit?
#         'gamma': 0,
        'max_depth': 10,
        'n_estimators': 100,
        #'random_state': 850564,
        'min_child_weight': 1,
        'objective': 'binary:logistic',
#                 'eta': 0.02, #use 0.002
        'eval_metric': 'logloss',
        'seed': i,
        'silent': True,
    #     booster='gbtree',
    #     n_jobs=1, 
        'nthread': 2,
    #     max_delta_step=0, 
    #     subsample=1, 
    #     colsample_bytree=1,
    #     colsample_bylevel=1, 
    #     reg_alpha=0,
    #     reg_lambda=1, 
    #     scale_pos_weight=1,
    #     base_score=0.5,
    #     random_state=0,
    #     seed=None, 
    #     missing=None, 
    }
    x1, x2, y1, y2 = train_test_split(X, y, test_size=0.33, random_state=i)
    watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]
    model = xgb.train(params, xgb.DMatrix(x1, y1), 150,  watchlist, feval=xgb_score, maximize=False, verbose_eval=50, early_stopping_rounds=50)
    if round != 0:
        pred += model.predict(xgb.DMatrix(aggtest[categorical_features+continuous_features]), ntree_limit=model.best_ntree_limit)
    else:
        pred = model.predict(xgb.DMatrix(aggtest[categorical_features+continuous_features]), ntree_limit=model.best_ntree_limit)
    round += 1
pred /= fold
test['is_churn'] = pred.clip(0.0000001, 0.999999)

[0]	train-log_loss:0.652451	valid-log_loss:0.652572
Multiple eval metrics have been passed: 'valid-log_loss' will be used for early stopping.

Will train until valid-log_loss hasn't improved in 50 rounds.
[50]	train-log_loss:0.153515	valid-log_loss:0.159285
[100]	train-log_loss:0.122428	valid-log_loss:0.134232
[0]	train-log_loss:0.652474	valid-log_loss:0.652575
Multiple eval metrics have been passed: 'valid-log_loss' will be used for early stopping.

Will train until valid-log_loss hasn't improved in 50 rounds.
[50]	train-log_loss:0.153462	valid-log_loss:0.15925
[100]	train-log_loss:0.122615	valid-log_loss:0.134184
[0]	train-log_loss:0.652453	valid-log_loss:0.652557
Multiple eval metrics have been passed: 'valid-log_loss' will be used for early stopping.

Will train until valid-log_loss hasn't improved in 50 rounds.
[50]	train-log_loss:0.153626	valid-log_loss:0.159135
[100]	train-log_loss:0.122941	valid-log_loss:0.134148


In [26]:
test.to_csv('../../results/xgbsubmission.csv')